In [1]:
import hvplot.pandas
import pandas as pd
import requests

In [2]:
from api_keys import geoapify_key

In [3]:
city_data_df = pd.read_csv(r"C:\Users\nicho\OneDrive\Desktop\Starter_Code (3)\Starter_Code\output_data\cities.csv")


In [4]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.53,66,0,1.79,US,1690295495
1,1,hami,42.8000,93.4500,32.64,10,0,6.69,CN,1690295738
2,2,west island,-12.1568,96.8225,24.99,73,40,8.23,CC,1690295667
3,3,yicheng,31.3705,119.8703,27.62,88,100,5.51,CN,1690295739
4,4,la passe,45.5549,-0.8967,24.13,56,11,7.94,FR,1690295739
...,...,...,...,...,...,...,...,...,...,...
595,595,igarka,67.4667,86.5833,20.44,67,6,3.89,RU,1690295663
596,596,gloucester,51.8657,-2.2431,20.76,53,75,0.89,GB,1690295913
597,597,puerto aysen,-45.4000,-72.7000,1.02,100,100,0.00,CL,1690295940
598,598,baykalovo,57.3981,63.7684,20.58,61,40,4.02,RU,1690295941


In [5]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.53,66,0,1.79,US,1690295495
1,1,hami,42.8000,93.4500,32.64,10,0,6.69,CN,1690295738
2,2,west island,-12.1568,96.8225,24.99,73,40,8.23,CC,1690295667
3,3,yicheng,31.3705,119.8703,27.62,88,100,5.51,CN,1690295739
4,4,la passe,45.5549,-0.8967,24.13,56,11,7.94,FR,1690295739


In [6]:
map_plot_1 = city_data_df.hvplot.points(
    "Lat",
    "Humidity",
    geo = True,
    grid = True,
    title = "City Latitude vs. Humidity (2022-10-18)",
    xlabel = "Latitude",
    ylabel = "Humidity(%)",
    frame_width = 600,
    frame_height = 500,
    line_width = 5
)

title = "City Latitude vs. Humidity (2022-10-18)"
xlabel = "Latitude"
ylabel = "Humidity (%)"

map_plot_1

C:\Users\nicho\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Points   [Lat,Humidity]

In [10]:
city_drop = []

for i in range(len(city_data_df)):
    if city_data_df["Cloudiness"][i] > 0:
        city_drop.append(i)
    
vacation_spot = city_data_df.drop(city_drop)
vacation_spot.dropna(inplace = True)



In [11]:
vacation_spot.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.53,66,0,1.79,US,1690295495
1,1,hami,42.8000,93.4500,32.64,10,0,6.69,CN,1690295738
26,26,port lincoln,-34.7333,135.8667,11.87,81,0,3.68,AU,1690295744
27,27,olhao,37.0286,-7.8411,29.49,62,0,4.63,PT,1690295678
38,38,bredasdorp,-34.5322,20.0403,12.87,68,0,2.70,ZA,1690295685
